# 🧪 ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

> **Note**: This notebook assumes that the agent files are stored in the `app` folder. If your agent files are located in a different directory, please update all relevant file paths and references accordingly.

## Install dependencies

In [ ]:
!pip install google-cloud-aiplatform google-cloud-trace --upgrade

### Initialize Vertex AI Client

In [1]:
# Initialize the Vertex AI client
#LOCATION = "asia-northeast1"
import json
import vertexai
LOCATION = "us-central1"
PROJECT_ID = "sandbox-373102"
# Set to None to auto-detect from ./deployment_metadata.json, or specify manually
#AGENT_ENGINE_ID = None
AGENT_ENGINE_ID = "5491845076462075904"
if AGENT_ENGINE_ID != None:
    REASONING_ENGINE_ID = f"projects/{PROJECT_ID}/locations/{LOCATION}/reasoningEngines/{AGENT_ENGINE_ID}"
else:
    try:
        with open("../deployment_metadata.json") as f:
            metadata = json.load(f)
            REASONING_ENGINE_ID = metadata.get("remote_agent_engine_id")
    except (FileNotFoundError, json.JSONDecodeError):
        pass
    
client = vertexai.Client(
    location=LOCATION,
)

C:\Users\jeehyeok\AppData\Roaming\Python\Python312\site-packages\google\cloud\aiplatform\models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


## Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [2]:
print(f"Using REASONING_ENGINE_ID: {REASONING_ENGINE_ID}")
# Get the existing agent engine
remote_agent_engine = client.agent_engines.get(name=REASONING_ENGINE_ID)

Using REASONING_ENGINE_ID: projects/sandbox-373102/locations/us-central1/reasoningEngines/5491845076462075904


Generate test session

In [3]:
session = await remote_agent_engine.async_create_session(user_id="test")
session

{'id': '3458657036558925824',
 'userId': 'test',
 'appName': '5491845076462075904',
 'lastUpdateTime': 1762415608.231875,
 'events': [],
 'state': {}}

Send test query #1

In [4]:
async for event in remote_agent_engine.async_stream_query(
    message="hi! what's the weather in San francisco?", user_id="test", session_id = session['id']
):
    print(event)

{'content': {'parts': [{'thought_signature': 'CpkCAePx_14yl36WM9oim0QtN2L6QDv6zRol0-NfEOeyTLxyfh_P78G80SnhGlvlwpr1ahoZijYF3-ZWloR27yoqvTtfaKJstwxePJEDuYfGepk-OID71s2DxebVAnOpvWGl5rKcmNDC16OpgAixyNBU9hLi94ZVzgtNBVhmEGYeu_sBVKjC6vDGLTtJt7ouo9UmSZHEYeygxYEGxLj3gTslSFH0NeYe04VFX5T3mSYjj9fa0OHtNF4dw8jg1_t7Ocn1w7aMGzbsEtGOrOX4OS4oKijevgsdSvcz1uUV0e0DcJtwZcVYBbmuTUMqkRpjN59q9Rpc-IUn_4x3Dkr2op5SaslGbCq4sdAT5OPV-IpSy4475HQBhROQJD0=', 'function_call': {'id': 'adk-559ca127-7a20-4892-b7f4-0e008c5d4c4c', 'args': {'query': 'San Francisco'}, 'name': 'get_weather'}}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 6, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 6}], 'prompt_token_count': 156, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 156}], 'thoughts_token_count': 57, 'total_token_count': 219, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -1.244869073232015, 'invocation_id': 'e-4ef7e533-cb2d-4169-8bb2-481625162f58', 'aut

Send test query #2

In [5]:
async for event in remote_agent_engine.async_stream_query(
    message="hi! what's the weather in chicago?", user_id="test", session_id = session['id']
):
    print(event)

{'content': {'parts': [{'thought_signature': 'Co0CAePx_14XTMiOqV_Zm4psj5EH9VQndqGlSBUGNA0loGPOJ_MKxPxkkEqztDFl69kMPEarwXau5h5hLMlEqZ_SlJHLwGISZwVh2y862ZBGq8lBiNq8UNr95o_dcT4DYw5nOHw81n7C46GkqgNgWuAGNyiHEXA0KxFBVx1pvNRl3wn4TFpr95y_YM8e23YmyTcMPZkQAXJDOEFmyHOTZ9-ywZeH9NdoF3wJ4iDYRLkYfEn5hRY2CMz4EZfriEGp3q3PW84iknkY0Pwe2eAwVU5S60KAplHU6vgVyp75imTxElNqaCJQp8Xjth3oFHE9WbfllvGDkup3BiDhDu9YxesTqVM0heb1VeuhqBMbzpE=', 'function_call': {'id': 'adk-22828ffb-9142-47d6-8952-516fc5fe6cee', 'args': {'query': 'Chicago'}, 'name': 'get_weather'}}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 5, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 5}], 'prompt_token_count': 199, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 254}], 'thoughts_token_count': 55, 'total_token_count': 259, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.2066568374633789, 'invocation_id': 'e-cd09bae6-d968-46f9-b24b-a8250e746573', 'author': 'root_agent', '

Invoke custom function

In [6]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

Manually create memory (By default, memory is not automatically generated)

In [ ]:
session = await remote_agent_engine.async_get_session(user_id="test", session_id=session['id'])
await remote_agent_engine.async_add_session_to_memory(session=session)

Search memory using similarity search by query

In [ ]:
await remote_agent_engine.async_search_memory(user_id="test", query="San Francisco")

In [7]:
pager = client.agent_engines.memories.list(name=REASONING_ENGINE_ID)
for i, page in enumerate(pager):
  print(f"========== Memory {i} =========")
  print(f"Expire: {page.expire_time}")
  print(f"TTL: {page.ttl}")
  print(f"Revision Expire: {page.revision_expire_time}")
  print(f"Revision TTL: {page.revision_ttl}")
  print(f"Disable memory revisions: {page.disable_memory_revisions}")
  print(f"Create time: {page.create_time}")
  print(f"Description: {page.description}")
  print(f"Display name: {page.display_name}")
  print(f"Fact: {page.fact}")
  print(f"Name: {page.name}")
  print(f"Scope: {page.scope}")
  print(f"Update time: {page.update_time}")
  print("\n")


========== Memory 0 =========
Expire: None
TTL: None
Revision Expire: None
Revision TTL: None
Disable memory revisions: None
Create time: 2025-11-06 07:53:46.010871+00:00
Description: None
Display name: None
Fact: I am interested in the weather in Chicago.
Name: projects/1045259343465/locations/us-central1/reasoningEngines/5491845076462075904/memories/7602176776315338752
Scope: {'app_name': '5491845076462075904', 'user_id': 'test'}
Update time: 2025-11-06 07:53:46.010871+00:00


========== Memory 1 =========
Expire: None
TTL: None
Revision Expire: None
Revision TTL: None
Disable memory revisions: None
Create time: 2025-11-06 07:53:46.010049+00:00
Description: None
Display name: None
Fact: I am interested in the weather in San Francisco.
Name: projects/1045259343465/locations/us-central1/reasoningEngines/5491845076462075904/memories/2990490757887950848
Scope: {'app_name': '5491845076462075904', 'user_id': 'test'}
Update time: 2025-11-06 07:53:46.010049+00:00




Get Trace data (https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/tracing_agents_in_agent_engine.ipynb)

In [16]:
from google.cloud import trace_v1 as trace
client = trace.TraceServiceClient()
result = [
    r
    for r in client.list_traces(
        request=trace.types.ListTracesRequest(
            project_id=PROJECT_ID,
            # Return all traces containing `labels {key: "openinference.span.kind" value: "AGENT"}`
            #filter="trace_id:d5640adfabcb9f9678fb37f63471b0ae",
            filter="root:invocation",
            order_by="start desc" #Decending order
        )
    )
]
result

[project_id: "sandbox-373102"
 trace_id: "d5640adfabcb9f9678fb37f63471b0ae",
 project_id: "sandbox-373102"
 trace_id: "f62afe67c1cf5a33fb16d185b5d8fb77"]

In [17]:
client.get_trace(
    request=trace.types.GetTraceRequest(
            project_id=PROJECT_ID,
            trace_id = result[0].trace_id
        )
)

project_id: "sandbox-373102"
trace_id: "d5640adfabcb9f9678fb37f63471b0ae"
spans {
  span_id: 16420876515183475539
  name: "invocation"
  start_time {
    seconds: 1762415622
    nanos: 191543824
  }
  end_time {
    seconds: 1762415624
    nanos: 585397436
  }
  labels {
    key: "g.co/agent"
    value: "opentelemetry-python 1.37.0; google-cloud-trace-exporter 1.9.0"
  }
}
spans {
  span_id: 2837328425360917667
  name: "agent_run [root_agent]"
  start_time {
    seconds: 1762415622
    nanos: 539343517
  }
  end_time {
    seconds: 1762415624
    nanos: 585299689
  }
  parent_span_id: 16420876515183475539
  labels {
    key: "g.co/agent"
    value: "opentelemetry-python 1.37.0; google-cloud-trace-exporter 1.9.0"
  }
}
spans {
  span_id: 15968098269431560266
  name: "call_llm"
  start_time {
    seconds: 1762415622
    nanos: 735116612
  }
  end_time {
    seconds: 1762415623
    nanos: 630658781
  }
  parent_span_id: 2837328425360917667
  labels {
    key: "gen_ai.usage.output_tokens"


### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [ ]:
# Uncomment the following lines if you're not using the virtual environment created by uv
# import sys
# sys.path.append("../")

In [ ]:
import sys
sys.path.insert(0, '../')
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [ ]:
async for event in agent_engine.async_stream_query(message="hi!", user_id="test"):
    print(event)

Delete agent

In [ ]:
client = vertexai.Client(
    location=LOCATION,
)
client.agent_engines.delete(
    name=REASONING_ENGINE_ID,
    force=True, # Optional, if the agent has resources (e.g. sessions, memory)
)